In [1]:
import torch, torchvision
from torch.nn import Module,Sequential,Linear,Conv2d,BatchNorm2d,ReLU,MaxPool2d
from torch.utils.data import DataLoader
import pathlib
import torch.nn as nn
from torch.optim import Adam
from torchvision import transforms
import glob

In [2]:
t = torch.randn(4,5,7)

In [3]:
batch_size = 100
class ConvNet(nn.Module):
    def __init__(self,num_classes=3):
        #giving default number of classes as 3
        super(ConvNet,self).__init__()
        
        #Input shape= (batch_size,3,150,150)
        # batch_size images in a batch x 3 channels(r,g,b) in an image x (150*150) pixels in an image.
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Applies 12 different filters and therefore obtains 12 different activation maps for all images in the btatch so only depth is changed
        #Shape= (batch_size,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Number of features is only fed as the batchnorm input
        #Shape= (batch_size,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (batch_size,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (batch_size,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (batch_size,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (batch_size,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (batch_size,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (batch_size,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (batch_size,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
        #Above output will be in matrix form, with shape (batch_size,32,75,75)
            
        output=output.view(-1,32*75*75)
        #C-1 inferrs values from other dimensions to ensure the final dimension is equla to the previous end multiplication result(batch_size,32,75,75)
        #batch_size entries with each entry as a single arra flattened from previous matrices . Each array length = 32*75*75
            
            
        output=self.fc(output)
            
        return output

In [4]:

preprocess = transforms.Compose(
    [
        transforms.Resize(150),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ]
)

Changes to be made in below cell :

Make a way to slice the dataset for each client
Compile everything to one single class
So, take number of data samples and batch size as an input to the model class.

In [5]:

train_path = '/Users/tarunvisvar/Downloads/Dataset/Handwriting/Handwriting-subset/Train'
test_path = '/Users/tarunvisvar/Downloads/Dataset/Handwriting/Handwriting-subset/Test'

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform = preprocess),
    batch_size=batch_size, shuffle=True
)

test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform = preprocess),
    batch_size=batch_size, shuffle=True
) 

In [17]:
len(iter(test_loader))*batch_size

1100

In [6]:
root = pathlib.Path(train_path)
classes = [dir.name for dir in root.iterdir()]
classes.remove('.DS_Store')

In [7]:
classes

['Reversal', 'Normal', 'Corrected']

In [8]:
model = ConvNet(num_classes = 3)
optimizer = Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [9]:
#calculating the size of training and testing images

train_count=len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.png'))

In [10]:
train_count,test_count

(4000, 1040)

In [11]:
num_epochs = 10

In [12]:
best_accuracy = 0.0

for epoch in range(num_epochs):
     model.train()
     #Model will be in training mode and takes place on training dataset
     train_loss = 0.0
     train_accuracy = 0.0
     for images,labels in train_loader:
          # The loop runs for 'number of batches' times 
          optimizer.zero_grad()
          outputs = model(images) 
          # The images(in batches) are preprocessed while brought up by the trainloader itself
          # The images(in batches) are passed through various layers and predictions are made.
          # Those are the outputs and are compared with the labels
          # The output is a batch_size length vector containing predicted output for batch_size images

          loss = loss_func(outputs,labels)
          loss.backward() # backpropagation
          optimizer.step() # Updates the weights
          #print("loss.data = ",loss.data)
          
          # For each image, we must add the loss to training loss. But loss is given for a batch by the model
          # So, we take the loss for a batch and multiply it with the batch size to get the loss for each image in an approximate manner

          train_loss += loss.data*batch_size
          #print(outputs.data) #outputs will be of size 10 x 3, for 10 images in a batch and 3 predictions for each image in a batch
          _,predictions = torch.max(outputs.data,1)
          #print(predictions) #predictions will contain the indices of the highest value outputed for each image. Therefore, predictions will contain 10(batch_size) values of the indices(hence also the classes)
          train_accuracy+=int(torch.sum(predictions==labels.data))
     train_accuracy /= train_count
     train_loss /= train_count
     print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))
     model.eval()
     #Modle will eb in the mode on evaluating on test dataset
     test_accuracy = 0.0
     for images,labels in test_loader:
          outputs = model(images)
          _,predictions = torch.max(outputs.data,1)
          #print(outputs.data)
          test_accuracy += int(torch.sum(predictions==labels.data))
     test_accuracy /= test_count
     print("Test accuracy =  ",str(test_accuracy))
     if test_accuracy>best_accuracy:
        torch.save(model,'best_checkpoint.model')
        best_accuracy=test_accuracy

KeyboardInterrupt: 

In [ ]:
print(best_accuracy)

0.9596153846153846


In [ ]:
loaded_model = torch.load('best_checkpoint.model')

In [ ]:
for name,param in loaded_model.named_parameters():
    if param.requires_grad:
        print(name,param.grad)

conv1.weight None
conv1.bias None
bn1.weight None
bn1.bias None
conv2.weight None
conv2.bias None
conv3.weight None
conv3.bias None
bn3.weight None
bn3.bias None
fc.weight None
fc.bias None


In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

548399

Inference:

Number of parameters in a model = around 5 and half lakhs

In [ ]:

class HWRModel:
    def __init__(self,data_path,batch_size,local_data_count):
        self.batch_size = batch_size
        self.train_path = data_path + '/Train'
        self.test_path = data_path + '/Test'
        self.local_data_count = local_data_count # Amount of data that a user can choose 
    
    def preprocess(self,resize=150):
        transformer = transforms.Compose(
            [
                transforms.Resize(resize),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
            ]
        )
        return transformer   

    def get_model(self):
        model = ConvNet(num_classes = 3)
        optimizer = Adam(model.parameters(), lr=0.001)
        loss_func = nn.CrossEntropyLoss()

        return (model,optimizer,loss_func)
    
    #Add load dataset function.

    def load_dataset(self):
        train_loader = DataLoader(
    torchvision.datasets.ImageFolder(self.train_path,transform = self.preprocess()),
    batch_size=batch_size, shuffle=True)

        test_loader = DataLoader(
    torchvision.datasets.ImageFolder(self.test_path,transform = self.preprocess()),
    batch_size=batch_size, shuffle=True) 

        return(train_loader,test_loader)
        
    def train(self,num_epochs=10):
        model,optimizer,loss_func = self.get_model()
        best_accuracy = 0.0
        train_loader,test_loader = self.load_dataset()
        train_count=len(glob.glob(self.train_path+'/**/*.png'))
        test_count=len(glob.glob(self.test_path+'/**/*.png'))

        
        for epoch in range(num_epochs):
            model.train()
            #Model will be in training mode and takes place on training dataset
            train_loss = 0.0
            train_accuracy = 0.0
            for images,labels in train_loader:
                # The loop runs for 'number of batches' times 
                optimizer.zero_grad()
                outputs = model(images) 
                # The images(in batches) are preprocessed while brought up by the trainloader itself
                # The images(in batches) are passed through various layers and predictions are made.
                # Those are the outputs and are compared with the labels
                # The output is a batch_size length vector containing predicted output for batch_size images

                loss = loss_func(outputs,labels)
                loss.backward() # backpropagation
                optimizer.step() # Updates the weights
                #print("loss.data = ",loss.data)
                
                # For each image, we must add the loss to training loss. But loss is given for a batch by the model
                # So, we take the loss for a batch and multiply it with the batch size to get the loss for each image in an approximate manner

                train_loss += loss.data*batch_size
                #print(outputs.data) #outputs will be of size 10 x 3, for 10 images in a batch and 3 predictions for each image in a batch
                _,predictions = torch.max(outputs.data,1)
                #print(predictions) #predictions will contain the indices of the highest value outputed for each image. Therefore, predictions will contain 10(batch_size) values of the indices(hence also the classes)
                train_accuracy+=int(torch.sum(predictions==labels.data))
            train_accuracy /= train_count
            train_loss /= train_count
            print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))
            model.eval()
            #Modle will eb in the mode on evaluating on test dataset
            test_accuracy = 0.0
            for images,labels in test_loader:
                outputs = model(images)
                _,predictions = torch.max(outputs.data,1)
                #print(outputs.data)
                test_accuracy += int(torch.sum(predictions==labels.data))
            test_accuracy /= test_count
            print("Test accuracy =  ",str(test_accuracy))
            if test_accuracy>best_accuracy:
                torch.save(model,'best_checkpoint.model')
                best_accuracy=test_accuracy
            
    def get_parameters(self):
        loaded_model = torch.load('best_checkpoint.model')
        params = dict()
        for name,parameters in loaded_model.named_parameters():
            params[name] = parameters
        return params
        



In [ ]:
data_path = '/Users/tarunvisvar/Downloads/Dataset/Handwriting/Handwriting-subset'
batch_size = 100
local_data_count = 1000
mymodel = HWRModel(data_path,batch_size,local_data_count)

In [ ]:
mymodel.train(num_epochs = 10)

Epoch: 0 Train Loss: tensor(10.7369) Train Accuracy: 0.5585
Test accuracy =   0.4913461538461538
Epoch: 1 Train Loss: tensor(0.9976) Train Accuracy: 0.799
Test accuracy =   0.46923076923076923
Epoch: 2 Train Loss: tensor(0.5559) Train Accuracy: 0.83675
Test accuracy =   0.8769230769230769
Epoch: 3 Train Loss: tensor(0.3880) Train Accuracy: 0.8765
Test accuracy =   0.9807692307692307
Epoch: 4 Train Loss: tensor(0.4969) Train Accuracy: 0.8585
Test accuracy =   0.8278846153846153
Epoch: 5 Train Loss: tensor(0.5767) Train Accuracy: 0.86
Test accuracy =   0.5403846153846154
Epoch: 6 Train Loss: tensor(0.5643) Train Accuracy: 0.86275
Test accuracy =   0.8903846153846153
Epoch: 7 Train Loss: tensor(0.3784) Train Accuracy: 0.887
Test accuracy =   0.9307692307692308
Epoch: 8 Train Loss: tensor(0.4004) Train Accuracy: 0.89475
Test accuracy =   0.9288461538461539
Epoch: 9 Train Loss: tensor(0.3224) Train Accuracy: 0.9025
Test accuracy =   0.9442307692307692


In [ ]:
mymodel.get_parameters()

{'conv1.weight': Parameter containing:
 tensor([[[[ 0.1690,  0.1080,  0.0092],
           [ 0.1585,  0.0621, -0.0139],
           [ 0.0812,  0.1706,  0.1587]],
 
          [[ 0.1087,  0.0890, -0.1338],
           [ 0.0097,  0.1451,  0.1072],
           [-0.1238, -0.0284, -0.1168]],
 
          [[ 0.1852,  0.0462, -0.0201],
           [-0.0992, -0.0722, -0.0999],
           [ 0.0374,  0.0253,  0.1712]]],
 
 
         [[[ 0.1264,  0.0022, -0.0193],
           [ 0.0385,  0.1762,  0.1856],
           [-0.0569, -0.1628, -0.1351]],
 
          [[ 0.1566,  0.1380, -0.0630],
           [-0.0288, -0.1585,  0.1070],
           [ 0.1057,  0.1422, -0.0259]],
 
          [[-0.0477,  0.1349,  0.1011],
           [-0.1836,  0.0726, -0.1369],
           [ 0.0583, -0.1209, -0.1937]]],
 
 
         [[[ 0.0687,  0.0492, -0.1707],
           [-0.1670, -0.1787, -0.1919],
           [-0.0685, -0.0459,  0.0223]],
 
          [[ 0.1041,  0.0377, -0.0591],
           [-0.0703,  0.0399,  0.0466],
           [-0

In [6]:
import tenseal as ts
import torch

In [7]:
    def context():
        context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
        context.global_scale = pow(2, 40)
        context.generate_galois_keys()
        return context
context = context()

In [11]:
tensor1 = torch.ones(2,7,5).tolist()
tensor2 = (torch.ones(2,7,5)*2).tolist()
tensor1

[[[1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0]],
 [[1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.0, 1.0, 1.0]]]

In [10]:
e1 = ts.ckks_vector(context,tensor1)
e2 = ts.ckks_vector(context,tensor2)

ValueError: can only encrypt a vector

In [25]:
res = (e1+e2)/torch.tensor(4)

In [26]:
res.decrypt().tolist()

[[[0.7500001010717946,
   0.7500001003320794,
   0.7500001009309527,
   0.7500001009210692,
   0.7500001006628408],
  [0.7500001004724166,
   0.7500001007525392,
   0.7500001006204756,
   0.7500001006311938,
   0.7500001001151912],
  [0.7500001002614758,
   0.7500001003634575,
   0.7500001006781736,
   0.7500001004644928,
   0.7500001006616509],
  [0.7500001004289406,
   0.7500000999360765,
   0.7500001004811281,
   0.7500001006519728,
   0.7500001001827724],
  [0.7500001006188527,
   0.7500001002779706,
   0.7500001006492009,
   0.7500001001849537,
   0.7500001005577396],
  [0.7500001008058593,
   0.7500001007423871,
   0.750000100504571,
   0.750000100304499,
   0.7500001003951088],
  [0.750000100500891,
   0.750000100750282,
   0.750000100533334,
   0.7500001006834965,
   0.7500001005819599]],
 [[0.7500001006871776,
   0.7500001002942924,
   0.7500001008281463,
   0.7500001009770606,
   0.7500001008488538],
  [0.750000101080207,
   0.7500001011238765,
   0.7500001005462207,
   0.750

In [20]:
tens1 = ts.PlainTensor([1,2.5,3])

In [21]:
e1 = ts.ckks_tensor(context,tens1)
e1

In [22]:
e1.decrypt().tolist()

[1.0000000007035317, 2.5000000002879323, 3.0000000017075275]

In [23]:
pip install phe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 411.6 kB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [24]:
from phe import paillier

In [25]:
public_key,private_key = paillier.generate_paillier_keypair()

In [26]:
li = [1,2,3]

<PaillierPublicKey 11d8b32a14>

In [28]:
pip install syft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 285.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.9/288.9 kB 187.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.6/448.6 kB 137.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 336.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 396.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 343.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 342.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.9/349.9 kB 437.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 519.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.5 MB/s eta 0:00:00-:--:--
     ━━━━━━━━━━━━━

In [ ]:
pip install --upgrade protobuf

In [10]:
pip install syft.frameworks

ERROR: Could not find a version that satisfies the requirement syft.frameworks (from versions: none)
ERROR: No matching distribution found for syft.frameworks
Note: you may need to restart the kernel to use updated packages.


In [8]:
ts.generate_ckks_keys()

AttributeError: module 'tenseal' has no attribute 'generate_ckks_keys'

In [11]:
from syft import frameworks

ImportError: cannot import name 'frameworks' from 'syft' (/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/syft/__init__.py)

In [ ]:
import tenseal as ts

: 

In [ ]:
a = torch.zeros(10)
b = torch.zeros(10)


In [13]:
import syft.frameworks.tenseal

ModuleNotFoundError: No module named 'syft.frameworks'

In [2]:
pip install phe

  Using cached phe-1.5.0-py2.py3-none-any.whl (53 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch

     ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/137.9 MB 164.6 kB/s eta 0:10:33

In [ ]:
from phe import paillier
import torch

: 

In [ ]:
public_key,private_key = paillier.generate_paillier_keypair()

: 

NameError: name 'torch' is not defined